1. Скачать, используя api.hh.ru, 500 вакансий, содержащих в названии "Разработчик", с
указанием ЗП
- 1.1. Оставить только id, name, area.name, salary.from, salary.to, salary.gross,
salary.currency, employer.name
- 1.2. Используя средства pandas, сохранить эти вакансии в виде df.csv, df.json.
2. Создать пустые базы db1 и db2 в Sqlite3, c полями из 1.1
3. Наполнить базы, используя, соответственно считывание из csv и из json.
4. Создать пустую базу db3. Создать три таблицы - вакансии, города, и работодатели.
Нормализовать таблицу из 1.1, и разбить ее данные на 3 перечисленные таблицы.
5. Средствами SQL создать новую колонку очищенной зарплаты в db3, содержащую среднюю зп по полям from и to, пересчитанную в рубли и очищенную от налог (для простоты - 13%)
7. По таблицам db3 с помощью JOIN, GROUP BY, HAVING, ORDER BY, вывести по убыванию очищенной зарплаты список вакансий в городах Москва и
Ташкент
8. С помощью оконных функций (dense_rank) вывести в каждом городе вакансию с максимальной очищенной зп
9. С помощью оконных функций (lag, lead, next) для каждой вакансии вывести макс и мин очищенную зп

In [1]:
import pandas as pd
import numpy as np
import requests
import sqlite3

 1. Скачать, используя api.hh.ru, 500 вакансий, содержащих в названии "Разработчик", с указанием ЗП
 - 1.1. Оставить только id, name, area.name, salary.from, salary.to, salary.gross, salary.currency, employer.name

In [2]:
df = pd.DataFrame()

In [3]:
for i in range(5):
  URL=f'https://api.hh.ru/vacancies?text=Разработчик&search_field=name&per_page=100&only_with_salary=True&page={i}'
  js=requests.get(URL).json()['items']
  tmp=(pd.json_normalize(js)[['id','name','area.name','employer.name',
                             'salary.from','salary.to','salary.currency','salary.gross']]).set_index('id')
  df=pd.concat([df,tmp])

In [4]:
df.head()

,name,area.name,employer.name,salary.from,salary.to,salary.currency,salary.gross
id,,,,,,,
117114407,Frontend-разработчик,Ташкент,NDC-CO,6000000.0,10000000.0,UZS,True
117110836,Java-разработчик,Алматы,SIRIUS LTW,100000.0,200000.0,KZT,False
117114557,Backend-разработчик (Java Spring),Ташкент,NDC-CO,6000000.0,NaN,UZS,False
117068398,Java-разработчик,Ташкент,LUDITO,45000000.0,75000000.0,UZS,False
117022964,Frontend-разработчик junior,Ташкент,COMING SOON STUDIO,5000000.0,8000000.0,UZS,True


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 117114407 to 116705010
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             500 non-null    object 
 1   area.name        500 non-null    object 
 2   employer.name    500 non-null    object 
 3   salary.from      421 non-null    float64
 4   salary.to        319 non-null    float64
 5   salary.currency  500 non-null    object 
 6   salary.gross     500 non-null    bool   
dtypes: bool(1), float64(2), object(4)
memory usage: 27.8+ KB


- 1.2. Используя средства pandas, сохранить эти вакансии в виде df.csv, df.json.

In [6]:
df.to_csv('data/df.csv')
df.to_json('data/df.json', orient='records', lines=True)

In [7]:
with open('data/df.csv','r') as f:
  [print(x) for x in f.readlines(1000)[:3]]

id,name,area.name,employer.name,salary.from,salary.to,salary.currency,salary.gross

117114407,Frontend-разработчик,Ташкент,NDC-CO,6000000.0,10000000.0,UZS,True

117110836,Java-разработчик,Алматы,SIRIUS LTW,100000.0,200000.0,KZT,False



2. Создать пустые базы db1 и db2 в Sqlite3, c полями из 1.1

In [8]:
def create_db(db_name):
  query = '''CREATE TABLE IF NOT EXISTS vacancies (
      id INTEGER PRIMARY KEY,
      name TEXT,
      area_name TEXT,
      salary_from INTEGER,
      salary_to INTEGER,
      salary_gross BOOLEAN,
      salary_currency TEXT,
      employer_name TEXT
  )'''
  conn = sqlite3.connect(db_name)
  cursor = conn.cursor()
  cursor.execute(query)
  conn.commit()
  return conn

In [9]:
conn1 = create_db("data/db1")

In [10]:
conn2 = create_db("data/db2")

3. Наполнить базы, используя, соответственно считывание из csv и из json

In [11]:
df_csv = pd.read_csv("data/df.csv")
df_csv.to_sql('vacancies', conn1, if_exists='replace', index=False)

500

In [12]:
df_json = pd.read_json("data/df.json", orient='records', lines=True)
df_json.to_sql('vacancies', conn2, if_exists='replace', index=False)

500

4. Создать пустую базу db3. Создать три таблицы - вакансии, города, и работодатели.
Нормализовать таблицу из 1.1, и разбить ее данные на 3 перечисленные таблицы.


In [13]:
conn3 = sqlite3.connect("data/db3")
c3 = conn3.cursor()

c3.execute(f"DROP TABLE IF EXISTS cities")
c3.execute(f"DROP TABLE IF EXISTS employers")
c3.execute(f"DROP TABLE IF EXISTS vacancies")

c3.execute("""
CREATE TABLE cities (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)
""")

c3.execute("""
CREATE TABLE employers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)
""")

c3.execute("""
CREATE TABLE vacancies (
    id TEXT,
    name TEXT,
    city_id INTEGER,
    "salary.from" REAL,
    "salary.to" REAL,
    "salary.gross" BOOLEAN,
    "salary.currency" TEXT,
    employer_id INTEGER,
    FOREIGN KEY (city_id) REFERENCES cities (id),
    FOREIGN KEY (employer_id) REFERENCES employers (id)
)
""")

cities = df[["area.name"]].drop_duplicates().reset_index(drop=True)
cities["name"] = cities["area.name"]
cities = cities.drop(columns=["area.name"])
cities.to_sql("cities", conn3, if_exists="append", index=False)

employers = df[["employer.name"]].drop_duplicates().reset_index(drop=True)
employers["name"] = employers["employer.name"]
employers = employers.drop(columns=["employer.name"])
employers.to_sql("employers", conn3, if_exists="append", index=False)

df["city_id"] = df["area.name"].apply(
    lambda x: c3.execute("SELECT id FROM cities WHERE name = ?", (x,)).fetchone()[0]
)
df["employer_id"] = df["employer.name"].apply(
    lambda x: c3.execute("SELECT id FROM employers WHERE name = ?", (x,)).fetchone()[0]
)

vacancies = df[
    [
        "name",
        "city_id",
        "salary.from",
        "salary.to",
        "salary.gross",
        "salary.currency",
        "employer_id",
    ]
]
vacancies.to_sql("vacancies", conn3, if_exists="append", index=False)

conn3.commit()
conn3.close()

5. Средствами SQL создать новую колонку очищенной зарплаты в db3, содержащую среднюю зп по полям from и to, пересчитанную в рубли и очищенную от налога (для
простоты - 13%)

In [ ]:
conn3 = sqlite3.connect("data/db3")
c3 = conn3.cursor()

c3.execute("""
ALTER TABLE DROP IF EXISTS vacancies ADD COLUMN mean_salary REAL
""")

c3.execute("""
UPDATE vacancies
SET mean_salary = (
    CASE
        WHEN "salary.currency" = 'USD' THEN ((COALESCE("salary.from", 0) + COALESCE("salary.to", 0)) / 2) * 100 * 0.87
        WHEN "salary.currency" = 'EUR' THEN ((COALESCE("salary.from", 0) + COALESCE("salary.to", 0)) / 2) * 105 * 0.87
        WHEN "salary.currency" = 'KZT' THEN ((COALESCE("salary.from", 0) + COALESCE("salary.to", 0)) / 2) * 0.2 * 0.87
        ELSE ((COALESCE("salary.from", 0) + COALESCE("salary.to", 0)) / 2) * 0.87
    END
)
""")

conn3.commit()
conn3.close()

6. По таблицам db3 с помощью JOIN, GROUP BY, HAVING, ORDER BY, вывести по убыванию очищенной зарплаты список вакансий в городах Москва и Ташкент

In [ ]:
conn3 = sqlite3.connect("data/db3")
c3 = conn3.cursor()

query = """
SELECT vacancies.id, vacancies.name, cities.name as city_name, vacancies.mean_salary
FROM vacancies
JOIN cities ON vacancies.city_id = cities.id
WHERE cities.name IN ('Москва', 'Ташкент')
GROUP BY vacancies.id, vacancies.name, cities.name, vacancies.mean_salary
HAVING vacancies.mean_salary IS NOT NULL
ORDER BY vacancies.mean_salary DESC
"""

result = pd.read_sql_query(query, conn3)
conn3.close()

result

,id,name,city_name,mean_salary
0,None,Инженер-программист АСУ ТП по автоматизации,Ташкент,14355000.0
1,None,Middle Flutter developer,Ташкент,8265000.0
2,None,Frontend-разработчик,Ташкент,6525000.0
3,None,Fronted-разработчик / Верстальщик,Ташкент,4785000.0
4,None,Ведущий программист микроконтроллеров (Кипр),Москва,525262.5
...,...,...,...,...
204,None,Junior разработчик на ASP.NET Core,Москва,21750.0
205,None,Программист C++ Junior,Москва,21750.0
206,None,Программист-стажёр,Москва,14790.0
207,None,Программист-разработчик (Junior),Москва,13050.0


7. С помощью оконных функций (dense_rank) вывести в каждом городе вакансию с максимальной очищенной зп

In [ ]:
conn3 = sqlite3.connect("data/db3")
c3 = conn3.cursor()

query = """
WITH ranked_vacancies AS (
    SELECT
        vacancies.id,
        vacancies.name,
        cities.name AS city_name,
        vacancies.mean_salary,
        DENSE_RANK() OVER (PARTITION BY cities.name ORDER BY vacancies.mean_salary DESC) AS rank
    FROM vacancies
    JOIN cities ON vacancies.city_id = cities.id
)
SELECT id, name, city_name, mean_salary
FROM ranked_vacancies
WHERE rank = 1
"""

result = pd.read_sql_query(query, conn3)
conn3.close()

result

,id,name,city_name,mean_salary
0,None,Программист 1С-Senior/Руководитель отдела разр...,Алматы,330600.0
1,None,Senior Python Developer,Астана,261000.0
2,None,Программист,Барнаул,110490.0
3,None,Backend-разработчик Python (Junior+),Бердск,23925.0
4,None,Java-разработчик,Бишкек,228375.0
...,...,...,...,...
72,None,Backend-разработчик,Чебоксары,43500.0
73,None,Программист-наладчик,Челябинск,195750.0
74,None,Программист фронтенд,Череповец,78300.0
75,None,Администратор баз данных (ученик программиста),Шуя,17400.0


8. С помощью оконных функций (lag, lead, next) для каждой вакансии вывести макс и мин очищенную зп

In [ ]:
conn3 = sqlite3.connect("data/db3")
c3 = conn3.cursor()

query = """
SELECT id, name, city_name, mean_salary,
    MAX(mean_salary) OVER (PARTITION BY city_name) as max_salary,
    MIN(mean_salary) OVER (PARTITION BY city_name) as min_salary,
    LAG(mean_salary) OVER (PARTITION BY city_name ORDER BY mean_salary) as prev_salary,
    LEAD(mean_salary) OVER (PARTITION BY city_name ORDER BY mean_salary) as next_salary
FROM (
    SELECT vacancies.id, vacancies.name, cities.name as city_name, vacancies.mean_salary
    FROM vacancies
    JOIN cities ON vacancies.city_id = cities.id
) subquery
"""

result = pd.read_sql_query(query, conn3)
conn3.close()

result

,id,name,city_name,mean_salary,max_salary,min_salary,prev_salary,next_salary
0,None,Junior Web Developer,Алматы,15660.0,330600.0,15660.0,NaN,17400.0
1,None,Программист-разработчик (Junior),Алматы,17400.0,330600.0,15660.0,15660.0,30450.0
2,None,Frontend-разработчик,Алматы,30450.0,330600.0,15660.0,17400.0,39150.0
3,None,PHP-разработчик (middle),Алматы,39150.0,330600.0,15660.0,30450.0,60900.0
4,None,Strong Junior Backend разработчик,Алматы,60900.0,330600.0,15660.0,39150.0,65250.0
...,...,...,...,...,...,...,...,...
495,None,Программист-наладчик,Челябинск,195750.0,195750.0,19575.0,152250.0,NaN
496,None,Программист фронтенд,Череповец,78300.0,78300.0,78300.0,NaN,NaN
497,None,Администратор баз данных (ученик программиста),Шуя,17400.0,17400.0,17400.0,NaN,NaN
498,None,Frontend Developer / WEB (Middle),Ярославль,113100.0,174000.0,113100.0,NaN,174000.0
